# 0 Init

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/NUS_MSBA/CS5344/CS5344 project/dataset

/content/drive/MyDrive/NUS_MSBA/CS5344/CS5344 project/dataset


In [ ]:
!pip install mlxtend

In [ ]:
import csv
import os
import json
import argparse
import random
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter



from tqdm import tqdm

# # spelling
# from spellchecker import SpellChecker

# nltk
import nltk
from nltk.corpus import stopwords

# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize

# # language detector
# from langdetect import detect
# from langdetect import DetectorFactory
# DetectorFactory.seed = 0

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 'here_for,'smoke','drink', 'match_age'
union_meta = ['scam','username','age','gender','location','ethnicity','occupation','marital_status','children','religion','sexual_orientation','sex', 'description']

scam_meta = ['email']

tqdm.pandas()
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
all_profile = pd.read_csv("processed_data.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 2 Market Basket for Attributes

In [ ]:
attribute_col = ['username','scam','age_bin','gender','location','ethnicity','marital_status','children','religion','sexual_orientation']
basket_col = ['scam','age_bin','gender','location','ethnicity','marital_status','children','religion','sexual_orientation']

all_profile_attribute = all_profile[attribute_col]
all_profile_attribute.dropna(subset=['username'], inplace=True)
all_profile_attribute.set_index('username', inplace=True)

# scam and non-scam
all_profile_attribute_scam = all_profile_attribute[all_profile_attribute['scam'] == 1]
all_profile_attribute_nonscam = all_profile_attribute[all_profile_attribute['scam'] == 0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-6-9e542e687be6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_profile_attribute.dropna(subset=['username'], inplace=True)


In [ ]:
# change column value into "col name - value" format
def transform_row(row):
    return [f"{column} - {value}" for column, value in row.items() if pd.notna(value)]

all_profile_attribute_scam_b = all_profile_attribute_scam[basket_col].apply(transform_row, axis=1)
all_profile_attribute_nonscam_b = all_profile_attribute_nonscam[basket_col].apply(transform_row, axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
all_profile_attribute_scam_b

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


username
kaylabrandy19          [scam - 1, age_bin - 31-40, gender - female, l...
queenpam288            [scam - 1, age_bin - 21-30, gender - female, l...
victoriaia             [scam - 1, age_bin - 41-50, gender - female, l...
harlimosely            [scam - 1, age_bin - 31-40, gender - female, l...
martha1010             [scam - 1, age_bin - 31-40, gender - female, m...
                                             ...                        
ryandad8               [scam - 1, age_bin - 41-50, gender - male, loc...
marcusw670             [scam - 1, age_bin - 51-60, gender - male, loc...
marcusw670             [scam - 1, age_bin - 51-60, gender - male, loc...
lovelessmichaelklus    [scam - 1, age_bin - 41-50, gender - male, loc...
lovelessmichaelklus    [scam - 1, age_bin - 41-50, gender - male, loc...
Length: 6064, dtype: object

In [ ]:
def onehot_enc(df):
  encoder = TransactionEncoder().fit(df)
  onehot = encoder.transform(df)
  df = pd.DataFrame(onehot, columns = encoder.columns_)
  return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
all_profile_attribute_scam_enc = onehot_enc(all_profile_attribute_scam_b)
all_profile_attribute_nonscam_enc = onehot_enc(all_profile_attribute_nonscam_b)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Run Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(all_profile_attribute_scam_enc, min_support=0.2, use_colnames=True)

# Generate association rules
scam_rules_raw = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
scam_rules_raw = scam_rules_raw[scam_rules_raw['consequents'] == {'scam - 1'}]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Run Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(all_profile_attribute_nonscam_enc, min_support=0.2, use_colnames=True)

# Generate association rules
nonscam_rules_raw = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
nonscam_rules_raw = nonscam_rules_raw[nonscam_rules_raw['consequents'] == {'scam - 0'}]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# remove duplicates
scam_antecedents_set = set(scam_rules_raw['antecedents'])
nonscam_antecedents_set = set(nonscam_rules_raw['antecedents'])

scam_antecedents_set_processed = scam_antecedents_set - nonscam_antecedents_set
nonscam_antecedents_set_processed = nonscam_antecedents_set - scam_antecedents_set

scam_rules = scam_rules_raw[scam_rules_raw['antecedents'].isin(scam_antecedents_set_processed)]
nonscam_rules = nonscam_rules_raw[nonscam_rules_raw['antecedents'].isin(nonscam_antecedents_set_processed)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
scam_rules_top7 = scam_rules[['antecedents', 'consequents', 'support']].sort_values(by='support', ascending=False)
scam_rules_top7.to_csv("scam_rules_top7.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
nonscam_rules_top7 = nonscam_rules[['antecedents', 'consequents', 'support']].sort_values(by='support', ascending=False)[:7]
nonscam_rules_top7.to_csv("nonscam_rules_top7.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
scam_rules_top7 = [set(x) for x in scam_rules_top7['antecedents']]
nonscam_rules_top7 = [set(x) for x in nonscam_rules_top7['antecedents']]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 4 Validation

In [ ]:
seed_value = 99999
np.random.seed(seed_value)

sample_size = 500

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# samples_scam = pd.DataFrame(all_profile_attribute_scam_b.sample(n=sample_size, random_state=seed_value))
# samples_nonscam = pd.DataFrame(all_profile_attribute_nonscam_b.sample(n=sample_size, random_state=seed_value))

samples_scam = pd.DataFrame(all_profile_attribute_scam_b)
samples_nonscam = pd.DataFrame(all_profile_attribute_nonscam_b)

samples_scam

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
username,
kaylabrandy19,"[scam - 1, age_bin - 31-40, gender - female, l..."
queenpam288,"[scam - 1, age_bin - 21-30, gender - female, l..."
victoriaia,"[scam - 1, age_bin - 41-50, gender - female, l..."
harlimosely,"[scam - 1, age_bin - 31-40, gender - female, l..."
martha1010,"[scam - 1, age_bin - 31-40, gender - female, m..."
...,...
ryandad8,"[scam - 1, age_bin - 41-50, gender - male, loc..."
marcusw670,"[scam - 1, age_bin - 51-60, gender - male, loc..."
marcusw670,"[scam - 1, age_bin - 51-60, gender - male, loc..."


In [ ]:
def calculate_marks_row(row):
    sample = set(row[0])
    marks = 0

    for scam_rule in scam_rules_top7:
        l = len(scam_rule)
        if scam_rule <= sample:
            marks -= 0.5 * l

    for nonscam_rule in nonscam_rules_top7:
        l = len(nonscam_rule)
        if nonscam_rule <= sample:
            marks += 0.5 * l

    return marks

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
samples_scam['marks'] = samples_scam.progress_apply(calculate_marks_row, axis=1)
samples_nonscam['marks'] = samples_nonscam.progress_apply(calculate_marks_row, axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 50920/50920 [00:00<00:00, 85349.66it/s]


In [ ]:
combined_samples = pd.concat([samples_scam, samples_nonscam], ignore_index=False)
combined_samples.to_csv('fis_samples.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
